## 두 개의 투자 원칙을 하나의 class로 삽입

1. 클래스의 이름은 invest 상속은 X
2. 생성자 함수
- 매개변수 4개
    - 주식 데이터(_df)
    - 기준이 되는 컬럼 이름 (_col = 'Adj Close')
    - 투자의 시작 시간 (_start = '2019-01-01')
    - 투자의 종료 시간 (_end = datetime.now())
- 결측치, 양의 무한대, 음의 무한대를 제외
- 컬럼 중 Date 컬럼이 존재한다면 Date 컬럼을 인덱스로 변환
- index를 시계열 데이터로 변환
- index에 tz에 존재한다면 tz를 None 변경
- 시작 시간과 종료 시간은 시계열 데이터로 변환
    - 종료시간은 문자열인 경우에만 변환
- 기준이 컬럼(self.col), 시작시간(self.start), 종료시간(self.end)

3. buyandhold 함수
4. bollinger 함수
    - 매개변수
        - 신뢰구간(_num = 2)
        - 데이터 개수 (_cnt = 20)
    - 특정 컬럼(self.col) 제외한 나머지 컬럼을 제거하고 변수에 저장
    - 이동평균선, 상단밴드, 하단밴드 생성
    - 보유 내역 추가
    - 데이터프레임 되돌려준다
5. 수익률 계산 함수
    - 복사본 생성
    - rtn 컬럼에 1을 대입
    - 매수, 매도 시기를 확인하여 수익률 대입
    - rtn을 기준으로 누적 수익률을 acc_rtn 컬럼에 대입
    - 만들어진 데이터프레임과 최종수익률을 되돌려준다

In [96]:
from datetime import datetime
import pandas as pd
import numpy as np

class invest :
    
    # 생성자 함수
    def __init__ (
        self,
        _df,
        _col = 'Adj Close',
        _start = '2019-01-01',
        _end = datetime.now()
    ):
        
        self.col = _col
        flag = _df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
        _df = _df.loc[~flag]
        
        if 'Date' in _df.columns:
            _df.set_index('Date', inplace = True)
            
        _df.index = pd.to_datetime(_df.index, format='%Y-%m-%d')
            
        if _df.index.tz:
            _df.index = _df.index.tz_localize(None)
            
        try :
            self.start = datetime.strptime(_start, '%Y-%m-%d')

            if type(_end) == 'str':
                self.end = datetime.strptime(_end, '%Y-%m-%d')
            else:
                self.end = _end

        except:
            print('시작 시간과 종료 시간의 포멧은 YYYY-mm-dd 입니다.')
            print('클래스를 다시 생성합니다')
        
        self.df = _df
        
    def buyandhold(self):
        # copy
        result = self.df.copy()
        
        # 날짜 설정
        result = result.loc[self.start : self.end, [self.col]]
        
        # 일별 수익률
        result['Daily Rtn'] = (result[self.col].pct_change() + 1).fillna(1)
        
        # 누적 수익률
        result['acc_rtn'] = result['Daily Rtn'].cumprod()
        
        # return 데이터에 데이터프레임, 총 수익률
        return result, result.iloc[-1,2]
    
    def bollinger(self, _num = 2, _cnt = 20):
        result = self.df.copy()
        result = result[[self.col]]
        
        # 이동평균선, 상단밴드, 하단밴드
        result['center'] = result[self.col].rolling(_cnt).mean()
        result['up'] = result['center'] + (_num * result[self.col].rolling(_cnt).std())
        result['down'] = result['center'] - (_num * result[self.col].rolling(_cnt).std())
        
        # 보유 내역 추가
        # 보유 내역 컬럼을 생성 '' 대입
        result['trade'] = ''

        # 내역 추가 
        for idx in result.index:
            # 상단 밴드보다 기준이 되는 컬럼의 값이 크거나 같은 경우
            if result.loc[idx, self.col] - result.loc[idx, 'up'] >= _num:
                # 매수중인 경우 매도 // 보유중 아니면 유지
                # trade = ''
                result.loc[idx, 'trade'] = ''
            # 하단 밴드보다 기준이 되는 컬럼의 값이 작거나 같은 경우 
            elif result.loc[idx, 'down'] - result.loc[idx, self.col] >= _num:
                # 보유중이 아니면 매수 // 보유중이면 유지 
                # trade = 'buy'
                result.loc[idx, 'trade'] = 'buy'
            # 밴드 중간에 기준이 되는 컬럼의 값이 존재한다면
            else:
                # 보유중이라면 보유 유지
                if result.shift().loc[idx, 'trade'] == 'buy':
                    result.loc[idx, 'trade'] = 'buy'
                # 보유중이 아니라면 유지
                else:
                    result.loc[idx, 'trade'] = ''
                    
        rtn_result, acc_rtn = self.create_rtn(result)
        return rtn_result, acc_rtn
    
    def create_rtn(self, _df):
        result = _df.copy()
        result['rtn'] = 1
        
        # 수익율 생성 
        for idx in result.index:
            # 매수 
            if (result.shift().loc[idx, 'trade'] == '') & \
                (result.loc[idx, 'trade'] == 'buy'):
                buy = result.loc[idx, self.col]
                print(f"매수일 : {idx}, 매수가 : {buy}")
            # 매도
            elif (result.shift().loc[idx, 'trade'] == 'buy') & \
                (result.loc[idx, 'trade'] == ''):
                sell = result.loc[idx, self.col]
                print(f"매도일 : {idx}, 매도가 : {sell}")
                # 수익율 계산 
                rtn = sell / buy
                # 컬럼에 대입 
                result.loc[idx, 'rtn'] = rtn
                print(f"수익율 : {rtn}")
        
        # 누적 수익율 계산
        result['acc_rtn'] = result['rtn'].cumprod()
        
        # 최종 누적 수익율 변수 저장
        acc_rtn = result.iloc[-1, -1]
        return result, acc_rtn
        

In [97]:
df = pd.read_csv("../../data/csv/AAPL.csv")
invest_aapl = invest(df)

In [98]:
invest_aapl.col

'Adj Close'

In [99]:
new_df, rtn = invest_aapl.buyandhold()

In [100]:
new_df

,Adj Close,Daily Rtn,acc_rtn
Date,,,
2019-01-02,156.642365,1.000000,1.000000
2019-01-03,141.039642,0.900393,0.900393
2019-01-04,147.060516,1.042689,0.938830
2019-01-07,146.733185,0.997774,0.936740
2019-01-08,149.530380,1.019063,0.954597
...,...,...,...
2019-06-18,198.449997,1.023518,1.266899
2019-06-19,197.869995,0.997077,1.263196
2019-06-20,199.460007,1.008036,1.273346


In [102]:
boll_df, acc_rtn = invest_aapl.bollinger()
boll_df, acc_rtn

매수일 : 2013-01-24 00:00:00, 매수가 : 56.879379
매도일 : 2014-04-28 00:00:00, 매도가 : 77.353386
수익율 : 1.359954826511028


C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_14720\1869399811.py:114: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.359954826511028' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.loc[idx, 'rtn'] = rtn


매수일 : 2015-07-09 00:00:00, 매수가 : 111.978203
매도일 : 2015-10-23 00:00:00, 매도가 : 111.557602
수익율 : 0.9962439029317162
매수일 : 2016-04-27 00:00:00, 매수가 : 92.532272
매도일 : 2016-09-15 00:00:00, 매도가 : 110.584213
수익율 : 1.1950880553327383
매수일 : 2017-06-12 00:00:00, 매수가 : 141.047043
매도일 : 2017-10-30 00:00:00, 매도가 : 162.341568
수익율 : 1.1509746290817313
매수일 : 2018-02-02 00:00:00, 매수가 : 156.846741
매도일 : 2018-08-01 00:00:00, 매도가 : 198.47876
수익율 : 1.2654312020419984
매수일 : 2018-11-05 00:00:00, 매수가 : 199.263809
매도일 : 2019-01-30 00:00:00, 매도가 : 163.913071
수익율 : 0.8225932838611952
매수일 : 2019-05-13 00:00:00, 매수가 : 185.720001


(             Adj Close    center          up        down trade  rtn   acc_rtn
 Date                                                                         
 1980-12-12    0.410525       NaN         NaN         NaN        1.0  1.000000
 1980-12-15    0.389106       NaN         NaN         NaN        1.0  1.000000
 1980-12-16    0.360548       NaN         NaN         NaN        1.0  1.000000
 1980-12-17    0.369472       NaN         NaN         NaN        1.0  1.000000
 1980-12-18    0.380182       NaN         NaN         NaN        1.0  1.000000
 ...                ...       ...         ...         ...   ...  ...       ...
 2019-06-18  198.449997  185.4325  201.032574  169.832427   buy  1.0  1.939901
 2019-06-19  197.869995  185.9960  202.558154  169.433846   buy  1.0  1.939901
 2019-06-20  199.460007  186.8300  204.361771  169.298229   buy  1.0  1.939901
 2019-06-21  198.779999  187.7860  205.751400  169.820600   buy  1.0  1.939901
 2019-06-24  199.169998  188.7960  206.944862  170.6

## 할로윈 투자 전략
1. 11월 첫날(시가) 구매 -> 4월 마지막날(종가) 판매 (6개월)
2. 판매한 금액 / 구매한 금액
3. 누적 수익률

4. 시간에서 6개월 뒤 (12개)-> 비교

In [179]:
from datetime import datetime
import pandas as pd
from dateutil.relativedelta import relativedelta

In [180]:
df = pd.read_csv('../../data/csv/AMZN.csv', index_col = 'Date')
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1997-05-15,2.437500,2.500000,1.927083,1.958333,1.958333,72156000
1997-05-16,1.968750,1.979167,1.708333,1.729167,1.729167,14700000
1997-05-19,1.760417,1.770833,1.625000,1.708333,1.708333,6106800
1997-05-20,1.729167,1.750000,1.635417,1.635417,1.635417,5467200
1997-05-21,1.635417,1.645833,1.375000,1.427083,1.427083,18853200


In [181]:
df.index = pd.to_datetime(df.index)

In [182]:
df.loc['2000-11'].iloc[0,0] # Open

35.8125

In [183]:
df.loc['2000-04'].iloc[-1,3]

55.1875

In [184]:
rtn_list = []
for i in range(2000, 2011, 1):
    buy_mon = f"{i}-11"
    sell_mon = f"{i+1}-4"
    
    buy = df.loc[buy_mon].iloc[0, 0]
    
    sell = df.loc[sell_mon].iloc[-1]['Close']
    
    rtn_list.append(sell / buy)

In [185]:
rtn_list

[0.4406282722513089,
 2.3573447740112994,
 1.4927160506095758,
 0.7956204159784748,
 0.9456458227455924,
 0.8809106359541997,
 1.608444804394314,
 0.8960683418803419,
 1.4289263506273773,
 1.155402000492095,
 1.1906962698211543]

In [186]:
acc_rtn = 1
for i in rtn_list:
    acc_rtn *= i

acc_rtn

2.9115898553801114

In [187]:
start = datetime(2000,11,1)
start

datetime.datetime(2000, 11, 1, 0, 0)

In [188]:
start + relativedelta(month=5)

datetime.datetime(2000, 5, 1, 0, 0)

In [193]:

halloween_df = pd.DataFrame()

for i in range(2000, 2011):
    start = datetime(year=i, month=11, day=1)
    end = start + relativedelta(months=5)
    start = start.strftime('%Y-%m')
    end = end.strftime('%Y-%m')
    
    start_df = df.loc[start].head(1)
    end_df = df.loc[end].tail(1)
    halloween_df = pd.concat([halloween_df, start_df, end_df])
    
halloween_df = halloween_df[['Open','Close']]
halloween_df

,Open,Close
Date,,
2000-11-01,35.812500,37.375000
2001-04-30,15.630000,15.780000
2001-11-01,7.080000,6.950000
2002-04-30,16.129999,16.690001
2002-11-01,19.219999,19.799999
2003-04-30,28.600000,28.690001
2003-11-03,54.799999,56.740002
2004-04-30,46.060001,43.599998
2004-11-01,34.220001,35.099998


In [206]:
halloween_df['rtn'] = 1
for i in range(1,len(halloween_df),2):
    rtn = halloween_df.iloc[i]['Close'] / halloween_df.iloc[i-1]['Open']
    halloween_df.iloc[i,2] = rtn

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_14720\3936132464.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.4406282722513089' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  halloween_df.iloc[i,2] = rtn


In [207]:
halloween_df

,Open,Close,rtn,acc_rtn
Date,,,,
2000-11-01,35.812500,37.375000,1.000000,1.000000
2001-04-30,15.630000,15.780000,0.440628,2.391235
2001-11-01,7.080000,6.950000,1.000000,2.391235
2002-04-30,16.129999,16.690001,2.357345,1.030321
2002-11-01,19.219999,19.799999,1.000000,1.030321
2003-04-30,28.600000,28.690001,1.492716,0.713299
2003-11-03,54.799999,56.740002,1.000000,0.713299
2004-04-30,46.060001,43.599998,0.795620,0.878693
2004-11-01,34.220001,35.099998,1.000000,0.878693


In [208]:
halloween_df['acc_rtn'] = halloween_df['rtn'].cumprod()
halloween_df.tail(1)

,Open,Close,rtn,acc_rtn
Date,,,,
2011-04-29,194.380005,195.809998,1.190696,2.91159


## 할로윈 투자 전략 함수
- 매개변수
    - _df
    - _start = 2010
    - _end = datetime.now().now
    - _mon = 11
- 복사본 생성
- 컬럼에 Date가 존재한다면 Date를 인덱스로 변환
- 인덱스를 시계열 데이터로 변경
- 빈 데이터프레임 생성
- 반복문을 이용하여 시작년도부터 종료 년도까지 반복
    - 시작년도의 월을 포함한 시계열 데이터 생성 (buy_mon)
    - buy_mon sell_mon 시계열데이터에서 년-월의 포맷으로 문자열로 변경
    - buy_mon 으로 인덱스를 필터에서 첫번째 인덱스를 추출
    - sell_mon 으로 인덱스를 필터에서 마지막 인덱스를 추출
    - 나온 결과를 빈 데이터프레임에서 단순 행 결합
- 수익률 계산
- 누적수익률 계산
- 만들어진 데이터프레임과 총 누적수익률을 되돌려준다

In [242]:
def halloween(
    _df,
    _start = '2010',
    _end = datetime.now().year,
    _mon = 11
):
    result = _df.copy()
    if 'Date' in result.columns:
        result.set_index('Date', inplace = True)
    
    result.index = pd.to_datetime(result.index)

    result = result[['Open', 'Close']]
    df = pd.DataFrame()
    try:
        for i in range(int(_start), int(_end)):
            start = datetime(year=i, month=_mon, day=1)
            end = start + relativedelta(months=5)
            start = start.strftime('%Y-%m')
            end = end.strftime('%Y-%m')
            buy_mon = result.loc[start].head(1)
            sell_mon = result.loc[end].tail(1)
            df = pd.concat([df, buy_mon, sell_mon])
    except:
       pass 
    df['rtn'] = 1
    for i in range(1,len(df),2):
        rtn = df.iloc[i]['Close'] / df.iloc[i-1]['Open']
        df.iloc[i,2] = rtn
        
    df['acc_rtn'] = df['rtn'].cumprod()
    acc_rtn = df.iloc[-1,-1]
    return df, acc_rtn
        

In [270]:
ticker = 'MSFT'
_df = pd.read_csv(f"../../data/csv/{ticker}.csv")

result, acc_rtn = halloween(_df)
result

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_14720\1496480561.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9642857501594401' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i,2] = rtn


,Open,Close,rtn,acc_rtn
Date,,,,
2010-11-01,26.879999,26.950001,1.000000,1.000000
2011-04-29,26.549999,25.920000,0.964286,0.964286
2011-11-01,26.190001,25.990000,1.000000,0.964286
2012-04-30,31.980000,32.020000,1.222604,1.178940
2012-11-01,28.840000,29.520000,1.000000,1.178940
2013-04-30,32.560001,33.099998,1.147711,1.353082
2013-11-01,35.669998,35.529999,1.000000,1.353082
2014-04-30,40.400002,40.400002,1.132605,1.532507
2014-11-03,46.889999,47.439999,1.000000,1.532507


In [271]:
acc_rtn - 1

1.3348148721467932

In [272]:
all_list = []
for i in range(1, 13):
    all_list.append(halloween(_df, _mon = i))

for i in range(1,13):
    print(f"{i}월 : {all_list[i-1][1] - 1}")

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_14720\1496480561.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7514696031525278' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i,2] = rtn
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_14720\1496480561.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9091229274083455' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i,2] = rtn
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_14720\1496480561.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8157802919708029' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i,2] = rtn
C:\Users\SAMSUNG\Ap

1월 : 0.7674357460745174
2월 : 0.7522026188552162
3월 : 0.7793918421982209
4월 : 0.6054993696899722
5월 : 0.7974066176366061
6월 : 1.1228089356976518
7월 : 1.443053956146267
8월 : 1.010291132264022
9월 : 1.46006802494871
10월 : 1.4650879004085304
11월 : 1.3348148721467932
12월 : 1.2426329632942368


In [273]:
maxValue = -1
maxMonth = 0
for i in range(1,13):
    a, b = halloween(_df, _mon = i)
    if maxValue < b:
        maxValue = b
        maxMonth = i

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_14720\1496480561.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7514696031525278' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i,2] = rtn
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_14720\1496480561.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9091229274083455' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i,2] = rtn
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_14720\1496480561.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8157802919708029' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i,2] = rtn
C:\Users\SAMSUNG\Ap

In [274]:
print(f'{ticker}의 할로윈 수익률이 가장 높은 달은 {maxMonth}월, 수익률은 {round(maxValue-1,3) * 100}%') 

MSFT의 할로윈 수익률이 가장 높은 달은 10월, 수익률은 146.5%
